In [ ]:
!pip install stanza
import stanza
import sqlite3

In [ ]:
con = sqlite3.connect("proyectoPura.db")
cur = con.cursor()

query = """
SELECT Título_Obra FROM OBRAS
UNION ALL
SELECT Título_Complemento FROM COMPLEMENTOS
UNION ALL
SELECT Título_Sección FROM SECCIONES_PUBLICADAS
"""

cur.execute(query)
titles = cur.fetchall()
#print(titles)

titles_list = []
for title in titles:
  if title[0] is not None:
    titles_list.append(title[0])
  if title[0] is None:
    titles_list.append("NULL") #we take into account "NULL" titles, to keep the same structure as original DB.
print(titles_list)

con.close()

['Al Sr. D. R. P. Enviándole mis obras', 'La mujer piadosa (Fragmentos)', 'Desvarío', 'Soneto. "¡Gloria y pobreza! El inmortal Quintana..."', 'Balada. La despedida de una hija', 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet', 'En el álbum de la señora M…', 'La primavera', 'La perejilera', 'Un viaje a la Luna. (Continuación)', 'Al señor Don B. M.', 'A orillas del Manzanares. La súplica', 'Un viaje a la Luna. (Continuación)', 'Fragmentos de la Rosa de Alejandra', 'Gente morena', 'Un viaje a la Luna. (Continuación)', 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo', 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños', 'Costumbres del siglo XIX. I', 'Margarita. Novela bretona', 'Los escondidos y la tapada. Fragmentos', 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum', 'Costumbres del siglo XIX.II', 'Margarita. Novela bretona. (Continuación)', 'A la niña Margarita Aurora de Wilson y Serrano', 'A María', 'A su Al

In [ ]:
#Delimitation of titles (later recognizing them)
import re
'''def find_delimit_subtit(text):
  #subtitle_ = re.sub(r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*', r' | ', text)
  subtitle_ = re.sub(r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*', r'" | ', text)## N: He restituído la comilla de cierre que te habías perdido, igual es problemático para los que no ponían comillas... lo vamos viendo
  return subtitle_'''

def subtitle_format(matchobj):
  match = matchobj.group(0)
  if re.fullmatch(r'\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*-\s*"\s*', match):
    return ' | "'
  elif re.fullmatch(r'\s*-\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*\[\.\.\.\]\s*"\s*-\s*', match):
    submatch = re.fullmatch(r'(\s*)\[\.\.\.\]\s*"\s*-\s*', match)
    return submatch.group(1)+'[...]" | '

def formatting_subtitles(text):
    subtit_re = r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*'
    new_text = re.sub(subtit_re, subtitle_format, text)
    return new_text

encontrar_subtit = [formatting_subtitles(row) for row in titles_list]
list_titsub = [row for row in encontrar_subtit if " | " in row]

#only this exceptions do not adjust to pattern (2 rows):
excepcion_1 = """Piquer-Oda "Alzad vuestras"""
excepcion_2 = """Matrimonios por anuncios-Los baños de Biarritz"""
def subadhoc(matchobj):
  return ' "'+ matchobj.group(0)[1:]
for i, row in enumerate(encontrar_subtit):
  if excepcion_1 in row:
    encontrar_subtit[i] = row.replace('Piquer-Oda "Alzad', 'Piquer-Oda. "Alzad') #Mismo formato que algunos sonetos.
  if excepcion_2 in row:
    encontrar_subtit[i]= row.replace('-', ' | ')
    #encontrar_subtit[i] = re.sub(r'[^|]+\]"', subadhoc, encontrar_subtit[i])##N: añadido mío (con la definición de subadhoc)

print(encontrar_subtit[249])
print(encontrar_subtit[207])

Inauguración del teatro de D. José Piquer-Oda. "Alzad vuestras tumbas [...]"
Matrimonios por anuncios | Los baños de Biarritz | "Hace pocos días [...]"


In [ ]:
#we continue preparing each title to process it through model (tok,lemm,pos,depparse,ner)
import re

#def clean_per_title(title):
  #cleaned_text = re.sub(
    #r'\s*\(\s*(Continuación|Fragmentos?|Conclusión|Conclusiones?|Conclusion)\.?\s*\)\s*', #not now, mayber after processing
    #' ',
    #title)
  #cleaned_text = re.sub(r'\s*"\s*|\s*\[\s*\.\s*\.\s*\.\s*\]\s*|\s*\[\s*\u2026\s*\]\s*', ' ', cleaned_text)
  #return cleaned_text.strip()

#cleaned_rows = [clean_per_title(row) for row in encontrar_subtit]

#print(cleaned_rows)

cleaned_rows = [row.split(" | ") if " | " in row else [row] for row in encontrar_subtit]

for row in cleaned_rows:
  print(row, "\n")

['Al Sr. D. R. P. Enviándole mis obras'] 

['La mujer piadosa (Fragmentos)'] 

['Desvarío'] 

['Soneto. "¡Gloria y pobreza! El inmortal Quintana..."'] 

['Balada. La despedida de una hija'] 

['Un viaje a la Luna. Prólogo: La pesadilla de Mocquet'] 

['En el álbum de la señora M…'] 

['La primavera'] 

['La perejilera'] 

['Un viaje a la Luna. (Continuación)'] 

['Al señor Don B. M.'] 

['A orillas del Manzanares. La súplica'] 

['Un viaje a la Luna. (Continuación)'] 

['Fragmentos de la Rosa de Alejandra'] 

['Gente morena'] 

['Un viaje a la Luna. (Continuación)'] 

['Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo'] 

['A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños'] 

['Costumbres del siglo XIX. I'] 

['Margarita. Novela bretona'] 

['Los escondidos y la tapada. Fragmentos'] 

['A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum'] 

['Costumbres del siglo XIX.II'] 

['Margarita. Novela bretona. (Continuación)'] 

In [ ]:
def processing_1(text, nlp_model):
  doc = nlp_model(text)
  if text != "NULL":
    return [(word.text, word.lemma, word.upos, word.head, word.deprel) for sent in doc.sentences for word in sent.words]
  else:
    return [("NULL")]
nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner,depparse")

processed_titles_es = [[processing_1(el, nlp_es) for el in lista] for lista in cleaned_rows]

for element in processed_titles_es:
  print(element, "\n")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[[('A', 'a', 'ADP', 3, 'case'), ('el', 'el', 'DET', 3, 'det'), ('Sr.', 'sr.', 'PROPN', 7, 'obl'), ('D.', 'd.', 'PROPN', 3, 'flat'), ('R.', 'R.', 'PROPN', 3, 'flat'), ('P.', 'p.', 'PROPN', 3, 'flat'), ('Enviando', 'enviar', 'VERB', 0, 'root'), ('le', 'él', 'PRON', 7, 'obl:arg'), ('mis', 'mi', 'DET', 10, 'det'), ('obras', 'obra', 'NOUN', 7, 'obj')]] 

[[('La', 'el', 'DET', 2, 'det'), ('mujer', 'mujer', 'NOUN', 0, 'root'), ('piadosa', 'piadoso', 'ADJ', 2, 'amod'), ('(', '(', 'PUNCT', 5, 'punct'), ('Fragmentos', 'Fragmentos', 'PROPN', 2, 'appos'), (')', ')', 'PUNCT', 5, 'punct')]] 

[[('Desvarío', 'desvarío', 'NOUN', 0, 'root')]] 

[[('Soneto', 'soneto', 'PROPN', 0, 'root'), ('.', '.', 'PUNCT', 1, 'punct'), ('"', '"', 'PUNCT', 3, 'punct'), ('¡', '¡', 'PUNCT', 3, 'punct'), ('Gloria', 'gloria', 'PROPN', 0, 'root'), ('y', 'y', 'CCONJ', 5, 'cc'), ('pobreza', 'pobreza', 'NOUN', 3, 'conj'), ('!', '!', 'PUNCT', 3, 'punct'), ('El', 'el', 'DET', 3, 'det'), ('inmortal', 'inmortal', 'ADJ', 3, 'amod')

In [ ]:
def processing_2(text, nlp_model):
  doc = nlp_model(text)
  if text != "NULL":
    return [(ent.text, ent.type) for ent in doc.entities]
  else:
    return [("NULL")]
nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner,depparse")

processed_ent_es = [[processing_2(el, nlp_es) for el in lista] for lista in cleaned_rows]

for element in processed_ent_es:
  print(element, "\n")

processed_ent_es

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[[('Sr. D. R. P.', 'MISC')]] 

[[('Fragmentos', 'MISC')]] 

[[('Desvarío', 'MISC')]] 

[[('Soneto', 'PER'), ('inmortal Quintana...', 'MISC')]] 

[[('Balada', 'PER')]] 

[[('Luna', 'MISC'), ('Mocquet', 'PER')]] 

[[('señora M…', 'PER')]] 

[[]] 

[[]] 

[[('Luna', 'MISC')]] 

[[('Don B. M.', 'PER')]] 

[[('Manzanares', 'LOC')]] 

[[('Luna', 'MISC')]] 

[[('de la Rosa de Alejandra', 'PER')]] 

[[]] 

[[('Luna', 'MISC')]] 

[[('Sr. D. Agustín Esteban Collantes', 'MISC')]] 

[[('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')]] 

[[('I', 'PER')]] 

[[('Margarita', 'PER')]] 

[[]] 

[[('Señora doña M. Del Pilar Sinués de Marco', 'PER')]] 

[[('II', 'MISC')]] 

[[('Margarita', 'PER')]] 

[[('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')]] 

[[('María', 'PER')]] 

[[('Alteza Real', 'PER'), ('Príncipe de Asturias', 'PER')]] 

[[('Margarita', 'PER')]] 

[[]] 

[[('Soneto', 'PER')]] 

[[('SS. MM. II', 'PER')]] 

[[('El matrimonio de la Princesa real de Inglaterra [...]', 'MISC'

[[[('Sr. D. R. P.', 'MISC')]],
 [[('Fragmentos', 'MISC')]],
 [[('Desvarío', 'MISC')]],
 [[('Soneto', 'PER'), ('inmortal Quintana...', 'MISC')]],
 [[('Balada', 'PER')]],
 [[('Luna', 'MISC'), ('Mocquet', 'PER')]],
 [[('señora M…', 'PER')]],
 [[]],
 [[]],
 [[('Luna', 'MISC')]],
 [[('Don B. M.', 'PER')]],
 [[('Manzanares', 'LOC')]],
 [[('Luna', 'MISC')]],
 [[('de la Rosa de Alejandra', 'PER')]],
 [[]],
 [[('Luna', 'MISC')]],
 [[('Sr. D. Agustín Esteban Collantes', 'MISC')]],
 [[('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')]],
 [[('I', 'PER')]],
 [[('Margarita', 'PER')]],
 [[]],
 [[('Señora doña M. Del Pilar Sinués de Marco', 'PER')]],
 [[('II', 'MISC')]],
 [[('Margarita', 'PER')]],
 [[('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')]],
 [[('María', 'PER')]],
 [[('Alteza Real', 'PER'), ('Príncipe de Asturias', 'PER')]],
 [[('Margarita', 'PER')]],
 [[]],
 [[('Soneto', 'PER')]],
 [[('SS. MM. II', 'PER')]],
 [[('El matrimonio de la Princesa real de Inglaterra [...]', 'MISC

In [ ]:
processed_ent_es[:5]

[[[('Sr. D. R. P.', 'MISC')]],
 [[('Fragmentos', 'MISC')]],
 [[('Desvarío', 'MISC')]],
 [[('Soneto', 'PER'), ('inmortal Quintana...', 'MISC')]],
 [[('Balada', 'PER')]]]

In [ ]:
cleaned_rows[:5]

[['Al Sr. D. R. P. Enviándole mis obras'],
 ['La mujer piadosa (Fragmentos)'],
 ['Desvarío'],
 ['Soneto. "¡Gloria y pobreza! El inmortal Quintana..."'],
 ['Balada. La despedida de una hija']]

In [ ]:
from ast import Pass
cleaned_rows_copy = cleaned_rows.copy()


for index, row in enumerate(processed_ent_es):
  for sub_index, entidad in enumerate(row):
    for tuple_item in entidad:
      if tuple_item is None:
        pass
      if isinstance(tuple_item, str):
        pass
      if isinstance(tuple_item, tuple):
        entity_str = tuple_item[0]
        replacement_str = f"[({tuple_item[0]}, {tuple_item[1]})]"
        cleaned_rows_copy[index][sub_index] = re.sub(re.escape(entity_str), replacement_str, cleaned_rows_copy[index][sub_index])
        #except TypeError:
          #pass
        #except IndexError:
          #pass

for el in cleaned_rows_copy[:50]:
    print(el, "\n")

['Al [(Sr. D. R. P., MISC)] Enviándole mis obras'] 

['La mujer piadosa ([(Fragmentos, MISC)])'] 

['[(Desvarío, MISC)]'] 

['[(Soneto, PER)]. "¡Gloria y pobreza! El [(inmortal Quintana..., MISC)]"'] 

['[(Balada, PER)]. La despedida de una hija'] 

['Un viaje a la [(Luna, MISC)]. Prólogo: La pesadilla de [(Mocquet, PER)]'] 

['En el álbum de la [(señora M…, PER)]'] 

['La primavera'] 

['La perejilera'] 

['Un viaje a la [(Luna, MISC)]. (Continuación)'] 

['Al señor [(Don B. M., PER)]'] 

['A orillas del [(Manzanares, LOC)]. La súplica'] 

['Un viaje a la [(Luna, MISC)]. (Continuación)'] 

['Fragmentos [(de la Rosa de Alejandra, PER)]'] 

['Gente morena'] 

['Un viaje a la [(Luna, MISC)]. (Continuación)'] 

['Al excmo. [(Sr. D. Agustín Esteban Collantes, MISC)], en el día de su santo'] 

['A la niña [(Margarita Aurora de Wilson, PER)] y [(Serrano, PER)], el día de su cumpleaños'] 

['Costumbres del siglo X[(I, PER)]X. [(I, PER)]'] 

['[(Margarita, PER)]. Novela bretona'] 

['Los escon

In [ ]:
with open("/content/ner_with_tit-subtit.txt", "w", encoding="utf-8") as f:
    for el in cleaned_rows_copy:
        f.write(str(el) + "\n")

In [ ]:
cleaned_rows_copy = cleaned_rows.copy()

for i, titulo in enumerate(processed_ent_es):
  for j, componentes in enumerate(titulo):
    for tuple_item in componentes:
      if isinstance(tuple_item, tuple):
        entity_str = tuple_item[0]
        replacement_str = str(tuple_item)
        try:
          cleaned_rows_copy[i][j] = re.sub(re.escape(entity_str), replacement_str, cleaned_rows_copy[i][j])
        except TypeError:
          pass
        except IndexError:
          pass

for el in cleaned_rows_copy[:50]:
    print(el, "\n")

["Al ('('('('('('('Sr. D. R. P.', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC') Enviándole mis obras"] 

["La mujer piadosa (('('('('('('('Fragmentos', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC'))"] 

["('('('('('('('Desvarío', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC')"] 

['(\'(\'(\'(\'(\'(\'(\'Soneto\', \'PER\')\', \'PER\')\', \'PER\')\', \'PER\')\', \'PER\')\', \'PER\')\', \'PER\'). "¡Gloria y pobreza! El (\'(\'(\'(\'(\'(\'(\'inmortal Quintana...\', \'MISC\')\', \'MISC\')\', \'MISC\')\', \'MISC\')\', \'MISC\')\', \'MISC\')\', \'MISC\')"'] 

["('('('('('('('Balada', 'PER')', 'PER')', 'PER')', 'PER')', 'PER')', 'PER')', 'PER'). La despedida de una hija"] 

["Un viaje a la ('('('('('('('Luna', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC')', 'MISC'). Prólogo: La pesadilla de ('('('('('('('Mocquet', 'PER')', 'PER')', 'PER')', 'PER')', 'PER')', 'PER')', 'PER')"] 

["En el álbum de la ('('('('('('('señora M

In [ ]:
for i, lista in enumerate(cleaned_rows):
  for element in lista:
    if element == "NULL":
      continue
    else:
      for tit_index, tit_content in enumerate(processed_ent_es):
        for ent in tit_content:
          for ner in ent:
            if not ner:
              continue
            if ner == "NULL":
                continue
            else:
              ent_str = ner[0]
              for per in ent_str:
                if per in element in lista in cleaned_rows:
                  copy_clean_rows = cleaned_rows.copy()
                  copy_clean_rows[i] = element.replace(per, f"{ent}")
copy_clean_rows





[['Al Sr. D. R. P. Enviándole mis obras'],
 ['La mujer piadosa (Fragmentos)'],
 ['Desvarío'],
 ['Soneto. "¡Gloria y pobreza! El inmortal Quintana..."'],
 ['Balada. La despedida de una hija'],
 ['Un viaje a la Luna. Prólogo: La pesadilla de Mocquet'],
 ['En el álbum de la señora M…'],
 ['La primavera'],
 ['La perejilera'],
 ['Un viaje a la Luna. (Continuación)'],
 ['Al señor Don B. M.'],
 ['A orillas del Manzanares. La súplica'],
 ['Un viaje a la Luna. (Continuación)'],
 ['Fragmentos de la Rosa de Alejandra'],
 ['Gente morena'],
 ['Un viaje a la Luna. (Continuación)'],
 ['Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo'],
 ['A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños'],
 ['Costumbres del siglo XIX. I'],
 ['Margarita. Novela bretona'],
 ['Los escondidos y la tapada. Fragmentos'],
 ['A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum'],
 ['Costumbres del siglo XIX.II'],
 ['Margarita. Novela bretona. (Continuación)']

In [ ]:
cleaned_rows_copy = cleaned_rows.copy()

for i, titulo in enumerate(processed_ent_es):
  for j, componentes in enumerate(titulo):
    for tuple_item in componentes:
      if isinstance(tuple_item, tuple):
        entity_str = tuple_item[0]
          replacement_str = str(tuple_item)

          try:
            cleaned_rows_copy[i][j] = re.sub(re.escape(entity_str), replacement_str, cleaned_rows_copy[i][j])
          except TypeError:
            pass
          except IndexError:
            pass

for el in cleaned_rows_copy[:50]:
    print(el, "\n")

["('I', 'PER')"] 

["('I', 'PER')"] 

["('I', 'PER')"] 

["('('('Soneto', 'PER')', 'PER')', 'PER')"] 

["('('('Balada', 'PER')', 'PER')', 'PER')"] 

["('I', 'PER')"] 

["('('('señora M…', 'PER')', 'PER')', 'PER')"] 

['La primavera'] 

['La perejilera'] 

["('I', 'PER')"] 

["('('('Don B. M.', 'PER')', 'PER')', 'PER')"] 

["('('('Manzanares', 'LOC')', 'LOC')', 'LOC')"] 

["('I', 'PER')"] 

["('I', 'PER')"] 

['Gente morena'] 

["('I', 'PER')"] 

["('I', 'PER')"] 

["('Margarita', 'PER')"] 

["('('('I', 'PER')', 'PER')', 'PER')"] 

["('('('Margarita', 'PER')', 'PER')', 'PER')"] 

["('I', 'PER')"] 

["('Señor', 'PER')"] 

["('I', 'PER')"] 

["('('('Margarita', 'PER')', 'PER')', 'PER')"] 

["('Margarita', 'PER')"] 

["('('('María', 'PER')', 'PER')', 'PER')"] 

["('('('Alteza Real', 'PER')', 'PER')', 'PER')"] 

["('('('Margarita', 'PER')', 'PER')', 'PER')"] 

['El otoño'] 

["('('('Soneto', 'PER')', 'PER')', 'PER')"] 

["('I', 'PER')"] 

["('I', 'PER')", '"Hace algunos años que un banquero

In [ ]:
import re

# Create a copy to store modifications
cleaned_rows_copy = [el if isinstance(el, str) else el[0] for row in cleaned_rows for el in row ]
#cleaned_rows_copy = cleaned_rows.copy()

for i, ner_list in enumerate(processed_ent_es):
  for entity_tuple in ner_list:
    if isinstance(entity_tuple, tuple):  # Ensure it's a tuple
      entity_text = entity_tuple[0]
      replacement_text = str(entity_tuple)

        cleaned_rows_copy = [re.sub(re.escape(entity_text), replacement_text, item) if isinstance(item, str) else item for item in cleaned_rows_copy]
        #cleaned_rows_copy[i] = re.sub(re.escape(entity_text), replacement_text, str(cleaned_rows_copy[i]))

for element in cleaned_rows_copy[:90]:
  print(element, "\n")

('I', 'PER') 

('I', 'PER') 

('I', 'PER') 

('Soneto', 'PER') 

('Balada', 'PER') 

('I', 'PER') 

('señora M…', 'PER') 

La primavera 

La perejilera 

('I', 'PER') 

('Don B. M.', 'PER') 

('Manzanares', 'LOC') 

('I', 'PER') 

('I', 'PER') 

Gente morena 

('I', 'PER') 

('I', 'PER') 

('Margarita', 'PER') 

('I', 'PER') 

('Margarita', 'PER') 

('I', 'PER') 

('Señor', 'PER') 

('I', 'PER') 

('Margarita', 'PER') 

('Margarita', 'PER') 

('María', 'PER') 

('Alteza Real', 'PER') 

('Margarita', 'PER') 

El otoño 

('Soneto', 'PER') 

('I', 'PER') 

('I', 'PER') 

"Hace algunos años que un banquero muy rico [...]" 

"Varios periódicos franceses y del extranjero [...]" 

('Poesía', 'MISC') 

('Alejandro Dumas', 'PER') 

('Soneto', 'PER') 

Las apariencias engañan. Historia-cuento 

('SS.', 'MISC') 

"El sábado a la una [...]" 

('El príncipe Mirza-Ally-Kan-Bahadoor [...]', 'MISC') 

"Habláse mucho en los círculos aristocrático de un matrimonio [...]" 

El invierno 

La hada azul 

"

In [ ]:
cleaned_rows_copy = cleaned_rows.copy()

for i, titulo in enumerate(processed_ent_es):
  for componentes in titulo:
    for tuple_item in componentes:
      if isinstance(tuple_item, tuple):
        entity_str = tuple_item[0]
        for row in cleaned_rows:
          for element in row:
            if entity_str in element:
              replacement_str = f"{tuple_item}"
              cleaned_rows_copy[i] = re.sub(rf'\b{entity_str}\b', replacement_str, cleaned_rows_copy[i])

for el in cleaned_rows_copy:
  print(el, "\n")


TypeError: expected string or bytes-like object, got 'list'

In [ ]:
#for lista in cleaned_rows:
  #for sublist in lista:

for lista_ in processed_ent_es:
  for tit in cleaned_rows:
    for sublist_ in lista_:
      for componente in tit:
        for tupla in sublist_:
          if isinstance (tupla, tuple):
            for elements in tupla:
              print("\n", elements)
          #for tit in cleaned_rows:
            #for subtit in tit:
              #if elements in subtit:
                #subtit = subtit.replace(elements, f"{tupla}")
                #print(subtit)
#print(cleaned_rows)


 Sr. D. R. P.

 MISC

 Fragmentos

 MISC

 Desvarío

 MISC

 Soneto

 PER

 inmortal Quintana...

 MISC

 Balada

 PER

 Luna

 MISC

 Mocquet

 PER

 señora M…

 PER

 Luna

 MISC

 Don B. M.

 PER

 Manzanares

 LOC

 Luna

 MISC

 de la Rosa de Alejandra

 PER

 Luna

 MISC

 Sr. D. Agustín Esteban Collantes

 MISC

 Margarita Aurora de Wilson

 PER

 Serrano

 PER

 I

 PER

 Margarita

 PER

 Señora doña M. Del Pilar Sinués de Marco

 PER

 II

 MISC

 Margarita

 PER

 Margarita Aurora de Wilson

 PER

 Serrano

 PER

 María

 PER

 Alteza Real

 PER

 Príncipe de Asturias

 PER

 Margarita

 PER

 Soneto

 PER

 SS. MM. II

 PER

 El matrimonio de la Princesa real de Inglaterra [...]

 MISC

 Poesía

 MISC

 Oda

 PER

 Alejandro Dumas

 PER

 Soneto

 PER

 SS.

 MISC

 AA. [...

 MISC

 El príncipe Mirza-Ally-Kan-Bahadoor [...]

 MISC

 Ismail

 PER

 Leviatán

 MISC

 M. de Lamartine

 PER

 Isabel II

 PER

 Hamlet

 PER

 Malibran

 ORG

 Londres

 LOC

 Brasil

 ORG

 M. 

In [ ]:
#for lista in cleaned_rows:
  #for sublist in lista:

cleaned_rows_copy = cleaned_rows.copy()

for lista_ in processed_ent_es:
  for tit in cleaned_rows:
    for sublist_ in lista_:
      for componente in tit:
        for tupla in sublist_:
          if isinstance (tupla, tuple):
            for elements in tupla:
              if elements in componente:
                componente = componente.replace(elements, f"{tupla}")
                print(componente)
             # for i, tit in enumerate(cleaned_rows):
               # if elements in cleaned_rows[i]:
                #cleaned_rows_copy[i] = re.sub(rf'\b{elements}\b', f"{tupla}", cleaned_rows_copy[i])
#for el in cleaned_rows_copy:
  #print("\n", el)

          #for tit in cleaned_rows:
            #for subtit in tit:
              #if elements in subtit:
                #subtit = subtit.replace(elements, f"{tupla}")
                #print(subtit)
#print(cleaned_rows)

Streaming output truncated to the last 5000 lines.
('Oda', '('doña Emilia Serrano de Wilson', 'PER')')
('Periquito el Veleidoso', '('doña Emilia Serrano de Wilson', 'PER')')
('I', '('doña Emilia Serrano de Wilson', 'PER')')
('Soledad', '('doña Emilia Serrano de Wilson', 'PER')')
('Periquito el Veleidoso', '('doña Emilia Serrano de Wilson', 'PER')')
('Serrano', '('doña Emilia Serrano de Wilson', 'PER')')
('J. R.', '('doña Emilia Serrano de Wilson', 'PER')')
('Rolandseck', '('doña Emilia Serrano de Wilson', 'PER')')
('marquesa', '('doña Emilia Serrano de Wilson', 'PER')')
('I', '('doña Emilia Serrano de Wilson', 'PER')')
('Soneto', '('doña Emilia Serrano de Wilson', 'PER')')
('Mariquita', '('doña Emilia Serrano de Wilson', 'PER')')
('Bonaparte', '('doña Emilia Serrano de Wilson', 'PER')')
('marquesa', '('doña Emilia Serrano de Wilson', 'PER')')
('marquesa', '('doña Emilia Serrano de Wilson', 'PER')')
('Soneto', '('doña Emilia Serrano de Wilson', 'PER')')
('I', '('doña Emilia Serrano de W

In [ ]:
cleaned_rows_copy = cleaned_rows.copy()

for i, titulo in enumerate(processed_ent_es):
  for componentes in titulo:
    for tuples in componentes:
      entity_str = tuples[0]
      for i, tit in enumerate(cleaned_rows):
        if entity_str in cleaned_rows[i]:
          replacement_str = f"{tuples}"
          cleaned_rows_copy[i] = re.sub(rf'\b{entity_str}\b', replacement_str, cleaned_rows_copy[i])



TypeError: decoding to str: need a bytes-like object, tuple found

In [ ]:
#for lista in cleaned_rows:
  #for sublist in lista:

cleaned_rows_copy = cleaned_rows.copy()

for lista_ in processed_ent_es:
    for sublist_ in lista_:
        for tupla in sublist_:
          if isinstance (tupla, tuple):
            for elements in tupla:
              for i, tit in enumerate(cleaned_rows):
                if elements in cleaned_rows[i]:
                  cleaned_rows_copy[i] = re.sub(rf'\b{elements}\b', f"{tupla}", cleaned_rows_copy[i])

for el in cleaned_rows_copy[:50]:
  print(el, "\n")
#for el in cleaned_rows_copy:

             # for i, tit in enumerate(cleaned_rows):
               # if elements in cleaned_rows[i]:
                #cleaned_rows_copy[i] = re.sub(rf'\b{elements}\b', f"{tupla}", cleaned_rows_copy[i])
#for el in cleaned_rows_copy:
  #print("\n", el)

          #for tit in cleaned_rows:
            #for subtit in tit:
              #if elements in subtit:
                #subtit = subtit.replace(elements, f"{tupla}")
                #print(subtit)
#print(cleaned_rows)


TypeError: expected string or bytes-like object, got 'list'

In [ ]:
import re


cleaned_rows_copy = cleaned_rows.copy()


for i, ner_list in enumerate(processed_ent_es):
  for entity_tuple in ner_list:
    if isinstance(entity_tuple, tuple):
      entity_text = entity_tuple[0]
        replacement_text = str(entity_tuple)
          cleaned_rows_copy[i] = re.sub(re.escape(entity_text), replacement_text, cleaned_rows_copy[i])

for element in cleaned_rows_copy:
  print(element, "\n")


["('I', 'PER')"] 

["('I', 'PER')"] 

["('I', 'PER')"] 

["('Soneto', 'PER')"] 

["('Balada', 'PER')"] 

["('I', 'PER')"] 

["('señora M…', 'PER')"] 

['La primavera'] 

['La perejilera'] 

["('I', 'PER')"] 

["('Don B. M.', 'PER')"] 

["('Manzanares', 'LOC')"] 

["('I', 'PER')"] 

["('I', 'PER')"] 

['Gente morena'] 

["('I', 'PER')"] 

["('I', 'PER')"] 

["('Margarita', 'PER')"] 

["('I', 'PER')"] 

["('Margarita', 'PER')"] 

["('I', 'PER')"] 

["('Señor', 'PER')"] 

["('I', 'PER')"] 

["('Margarita', 'PER')"] 

["('Margarita', 'PER')"] 

["('María', 'PER')"] 

["('Alteza Real', 'PER')"] 

["('Margarita', 'PER')"] 

['El otoño'] 

["('Soneto', 'PER')"] 

["('I', 'PER')"] 

["('I', 'PER')", '"Hace algunos años que un banquero muy rico [...]"', '"Varios periódicos franceses y del extranjero [...]"'] 

["('Poesía', 'MISC')"] 

["('Alejandro Dumas', 'PER')"] 

["('Soneto', 'PER')"] 

['Las apariencias engañan. Historia-cuento'] 

["('SS.', 'MISC')", '"El sábado a la una [...]"', "('El pr

In [ ]:
with open("/content/experimenting.txt", "w", encoding="utf-8") as f:
    for element in cleaned_rows_copy:
        f.write(str(element) + "\n")